# GTEx Age Prediction Mini Project

Karl Tayeb

## Instructions

To run this you just need to open 'run.sh' and specify $MAT_DIR and $ANNOT_DIR which are the directories to look for the gzipped gene expression matrices for each tissue and the the directory to grab the annotation txt file respectively. It will then run you through the following:

1. Seperating the data into training and test data
2. Data processing/tuning over model parameters
3. Fitting models with tuned parameters over different # of training samples
5. Generating plots

Dependencies:
- numpy, scipy, pandas
- matplotlib, seaborn for plotting
- scikit-learn

Please note, I've only confirmed that this runs on MARCC. I spent more time then I care to admit trying to get all the dependencies to work on HHPC and I'm not using anything non-standard. I don't have access to the ugrad cluster so I figured this was my best option.

## Approach

Predict age from GTEx gene expression using LASSO. This was an appealing approach because the sparsity inducing L1 penalty automatically tells us which features are important. By keeping our genes as features, the results lend temselves readily to biological interperetation.

Our feature matrices X for each tissue are simply the gene expression for that tissue across all genes. We choose how to clean the data (discussed below) through cross validation.

Our response vector Y is simply the age by decade (e.g 50-59 = 50). If we had to represent a decade by any number, moving this up or down by a year will only change the intercept.


## Model Tuning and Data Processing

Because Lasso removes much of the thinking form feature selection, the focus was on cleaning the data properly. We correct for the effects of gender and consider correcting for up to the top 10 principal components of each tissue. In lieu of selecting features, we do need to tune the penality parameter which will determine how sparse our coefficients are.

For each tissue we select the number of principal components to correct and a setting of the penalty paramemter through 10-fold cross validation.

It is worth noting that we center genes and scale them to unit variance before PCA and before finally fitting the model. We scale before PCA so that the principal components are not dominated by higher variance genes. After correcting for gender and the k top PCs, we rescale the genes again. We do this because we want the L1 penalty to have the same impact across genes which is only true if they are all on the same scale.

A small point of trouble: When I correct for PCs and gender, I'm correcting with a linear model that accounts for the effects of gender and the top PCs on gene expression, FIT ON THE TRAINING SET. Then we apply that model to the validation/test set. These transformations are based on the principal components of the test set and a linear model fit on the test set. 

Now, if I want to scale and center the data, should I scale and center the validation set wrt to itself, or should I apply the same transformations I made on the training set? I figured I ought not treat these trasnformations differently than the other corrections I perform, so I did the latter.

For each tissue I selected the principal components to remove + penalty paramter by the model settings that had the highest mean R^2 on the validation set across 10 folds.

## Cross validation suggested not correcting for PCs

For all of the tissues except whole blood, cross validation scores were highest when removing no PCs. 

Settings found through cross validation:
- thyroid regressed pcs: 0 penalty: 1.25
- muscle_skeletal regressed pcs: 0 penalty: 1.25
- adibpose_subcutaneous regressed pcs: 0 penalty: 1.25
- whole_blood regressed pcs: 6 penalty: 0.625

This was somewhat suprising. It seems that removal/inclusing of a principal component had only incremental effects on the models performance on outside data. If we were to believe that our data has strong batch effects/technical covariates, we might expect removal of PCs to have significant impact on the generalization of the model, this does not seem to be the case.

If the first PC did capture a lot a variation due to age then removing 1+ PCs would hurt us because we are removing our signal. Even if we might improve by removing say, the 2nd and 3rd PC, it wouldn't compensate for the loss of predictive power from correcting the 1st. If I had time to run this again I would make sure that I am not removing PCs that make significant contribution to describing age related variance in gene expression.


## Biological Interperetaion

Lasso induces sparsity on our model coefficients, so we can look at the set of genes with nonzero coefficients. In fact, the choice of penalty parameter made our regression coefficients extremely sparse. In the final model thyroid, muscle, adipose and whole-blood had 62, 49, 52, and 143 non-zero coefficients respsectively. These gene sets are quite managable, we can and should inspect these genes.

## Results

Finally, using the tissue specific parameter/pcs selected above, I looked at the learning curve for each tissue, training 10 models each on randomly subsetted samples from the training set sized [25, 50, 75, 100, ...] . I fit the model with the true responses as well as permuted responses. This allows us to see if our model's performance on the test data is any better than "random". We evaluate the models and permuted models on the test set. We use all of the test set when looking at how well Lasso performs across training sets within a tissues. For comparison across tissue, we restrict evaluation to 70 samples per tissue in the test set (which is the smallest sized test set across tissues)

![figure1](fig1.png)

Figure 1. True(blue) and permuted(green) model performance (R^2 of test data) across different sizes of training set.

The more samples we include in traiing, the more predicting our model is of age on held out data. Strong evidence that muscle_skeletal tissue gene expression is predictive of age.

Note: the error bars are not particularly meaningful here. As we increase the size of the traning set, it means that there is more overlap from training set to training set. So when the size of the trianing set is 250, which is very nearly all the samples we have of course we won't see much difference.

![figure2](fig2.png)

Figure 2. Test set performance for each tissue across number of sampels.

Here is a look at the other tissues. While all the tissues have some predictive power, it looks like subcutaneous adipose and skeletal muscle are more predictive than thyroid and whole blood. Whole blood is clearly the loser here.

We tuned the penalty parameter on all of the training data. This is more the case on the right side of the plot where training set sizes are large. It is not clear to me to what extent this introduces bias into the analysis, however it is possible that since the penalty parameter was selected over a larger set of data, it is less optimal when the training sets are small.

To combat this uncertainty I also fitted models on permuted data. The perforance of the permuted model on the test set is consistently bad which makes me more comfortable in saying that each of the tissues have some predictvie power.

![scores_pc_by_tissue](scores_pc_by_tissue.png)

Here we plot the R2 value on held out data across cross validation for different number of PCs removed with the penalty parameter that was best for each tissue. As we can see, no choice of principal components causes big jumps in improvement.

Across all tissues it seems removing the first PC marginally worsens the models performance on held out data. In all but thyroid, we gradually recover model performance as we remove more PCs. Perhaps the first PC in the other tissues is capturing age.

Subcutaneous adipose has very consistent model performance from run to run holding the number of PCs we remove fixed. This is certainly not the case in skeletal muscle, where we see that the performance of the model is quite sensitive to the choice of training data. I'm not quite sure what to make of this